In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from sklearn import ensemble, tree, linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import GradientBoostingRegressor
np.random.seed(453111) # set the random seed

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
test_id =pd.read_csv("../data/test_id.csv", header=None)
Ames = pd.read_csv("../data/Ames_data.csv")
Ames_Location = Ames[["PID", "Latitude", "Longitude"]]
Ames_Location


,PID,Latitude,Longitude
0,526301100,42.054035,-93.619754
1,526350040,42.053014,-93.619756
2,526351010,42.052659,-93.619387
3,526353030,42.051245,-93.617320
4,527105010,42.060899,-93.638933
...,...,...,...
2925,923275080,41.988964,-93.604776
2926,923276100,41.988314,-93.602680
2927,923400125,41.986510,-93.606847
2928,924100070,41.990921,-93.600190


In [ ]:
# see column names
Ames.columns

Index(['PID', 'MS_SubClass', 'MS_Zoning', 'Lot_Frontage', 'Lot_Area', 'Street',
       'Alley', 'Lot_Shape', 'Land_Contour', 'Utilities', 'Lot_Config',
       'Land_Slope', 'Neighborhood', 'Condition_1', 'Condition_2', 'Bldg_Type',
       'House_Style', 'Overall_Qual', 'Overall_Cond', 'Year_Built',
       'Year_Remod_Add', 'Roof_Style', 'Roof_Matl', 'Exterior_1st',
       'Exterior_2nd', 'Mas_Vnr_Type', 'Mas_Vnr_Area', 'Exter_Qual',
       'Exter_Cond', 'Foundation', 'Bsmt_Qual', 'Bsmt_Cond', 'Bsmt_Exposure',
       'BsmtFin_Type_1', 'BsmtFin_SF_1', 'BsmtFin_Type_2', 'BsmtFin_SF_2',
       'Bsmt_Unf_SF', 'Total_Bsmt_SF', 'Heating', 'Heating_QC', 'Central_Air',
       'Electrical', 'First_Flr_SF', 'Second_Flr_SF', 'Low_Qual_Fin_SF',
       'Gr_Liv_Area', 'Bsmt_Full_Bath', 'Bsmt_Half_Bath', 'Full_Bath',
       'Half_Bath', 'Bedroom_AbvGr', 'Kitchen_AbvGr', 'Kitchen_Qual',
       'TotRms_AbvGrd', 'Functional', 'Fireplaces', 'Fireplace_Qu',
       'Garage_Type', 'Garage_Yr_Blt', 'Garage_Fi

In [ ]:
Ames.drop(['Garage_Yr_Blt',"Condition_2","Utilities","Roof_Matl","Latitude", "Longitude"], axis=1, inplace=True)
Ames.dropna(inplace=True)
Ames.head()


AttributeError: 'DataFrame' object has no attribute 'column'

In [ ]:
categorical = [var for var in Ames.columns if Ames[var].dtype=='O']
numerical = [var for var in Ames.columns if Ames[var].dtype!='O']
ordinal=["Overall_Qual", "Overall_Cond", "Exter_Qual", "Exter_Cond","Bsmt_Qual","Bsmt_Cond", "Heating_QC","Kitchen_Qual","Fireplace_Qu","Garage_Qual","Garage_Cond",
        "Pool_QC","Lot_Shape","Land_Slope","Bsmt_Exposure","BsmtFin_Type_1","BsmtFin_Type_2","Electrical","Functional",
    "Paved_Drive","Fence"]
nominal =[col for col in categorical if col not in ordinal]
out=Ames.Gr_Liv_Area.sort_values(ascending=False)[:3].index
Ames.drop(out, axis=0, inplace=True)
out=Ames.Total_Bsmt_SF.sort_values(ascending=False)[:3].index
Ames.drop(out, axis=0, inplace=True)

discrete = []
continous=[]
for var in numerical:
    if len(Ames[var].unique())<15:
        discrete.append(var)
    else:
        continous.append(var)
discrete.append("Year_Built")
discrete.append("Year_Remod_Add")
discrete.remove("BsmtFin_SF_1")
continous.remove("Year_Built")
continous.remove("Year_Remod_Add")
continous.append("BsmtFin_SF_1")

In [ ]:
Ames.replace({"Bsmt_Cond" : {"No_Basement" : 0, "Poor" : 1, "Fair" : 2, "Typical": 3, "Good" : 4, "Excellent" : 5},
                    "Bsmt_Exposure" : {"No_Basement":0,"No" : 1, "Mn" : 2, "Av": 3, "Gd" : 4},
                    "BsmtFin_Type_1" : {"No_Basement" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                        "ALQ" : 5, "GLQ" : 6},
                    "BsmtFin_Type_2" : {"No_Basement" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                        "ALQ" : 5, "GLQ" : 6},
                    "Alley": {"No_Alley_Access":0, "Gravel":1, "Paved":2},
                    "Street": {"Grvl":0, "Pave":1},
                    "Central_Air":{"Y":1, "N":0}, 
                    "Land_Contour": {"Bnk":0,"Lvl":1,"HLS":2, "Low":3},
                    "Bldg_Type":{"OneFam":0,"Duplex":1,"TwnhsE":2,"Twnhs":3,"TwoFmCon":4},
                    "Bsmt_Qual" : {"No_Basement" : 0, "Poor" : 1, "Fair" : 2, "Typical": 3, "Good" : 4, "Excellent" : 5},
                    "Exter_Cond" : {"Poor" : 1, "Fair" : 2, "Typical": 3, "Good": 4, "Excellent" : 5},
                    "Exter_Qual" : {"Poor" : 1, "Fair" : 2, "Typical": 3, "Good": 4, "Excellent" : 5},
                    "Fireplace_Qu" : {"No_Fireplace" : 0, "Poor" : 1, "Fair" : 2, "Typical" : 3, "Good" : 4, "Excellent" : 5},
                    "Functional" : {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5, 
                                    "Min2" : 6, "Min1" : 7, "Typ" : 8},
                                            "Garage_Cond" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                    "Garage_Qual" : {"No_Garage" : 0, "Poor" : 1, "Fair" : 2, "Typical" : 3, "Good" : 4, "Excellent" : 5},
                    "Heating_QC" : {"Poor" : 1, "Fair" : 2, "Typical": 3, "Good": 4, "Excellent" : 5},
                    "Garage_Finish":{"No_Garage":0, "Unf":1, "RFn":2, "Fin":3},
                    "Kitchen_Qual" : {"Poor" : 1, "Fair" : 2, "Typical": 3, "Good": 4, "Excellent" : 5},
                    "Land_Slope" : {"Sev" : 1, "Mod" : 2, "Gtl" : 3},
                    "Lot_Shape" : {"Irregular" : 1, "Moderately_Irregular" : 2, "Slightly_Irregular" : 3, "Regular" : 4},
                    "Paved_Drive" : {"Dirt_Gravel" : 0, "Paved" : 1, "Partial_Pavement" : 2},
                    "Pool_QC" : {"No_Pool" : 0, "Fair" : 1, "Typical" : 2, "Good" : 3, "Excellent" : 4},
                    "Overall_Qual":{"Very_Excellent": 10, "Excellent":9,"Very_Good":8, "Good":7, "Above_Average":6, "Average": 5, "Below_Average":4, "Fair": 3,"Poor":2, "Very_Poor":1},
                    "Overall_Cond" :{"Very_Excellent": 10, "Excellent":9,"Very_Good":8, "Good":7, "Above_Average":6, "Average": 5, "Below_Average":4, "Fair": 3,"Poor":2, "Very_Poor":1},
                    "Electrical ": {"Unknown": 0,"SBrkr ": 1,"FuseA": 2, "FuseF": 3, "FuseP": 4,"Mix": 5 },
                    "Fence":{"No_Fence":0, "Minimum_Wood_Wire":1,"Good_Wood":2,"Minimum_Privacy":3,"Good_Privacy":4},
                    "Garage_Cond" : {"No_Garage" : 0, "Poor" : 1, "Fair" : 2, "Typical" : 3, "Good" : 4, "Excellent" : 5},
                    }, inplace=True)

Ames["Electrical"].replace({"Unknown": 0,"SBrkr": 1,"FuseA": 2, "FuseF": 3, "FuseP": 4,"Mix": 5 }, inplace=True)

In [ ]:
Age_range = Ames['Year_Built'].max() - Ames['Year_Built'].min()
min_value = int(np.floor(Ames['Year_Built'].min()))
max_value = int(np.ceil(Ames['Year_Built'].max()))

# let's round the bin width
inter_value = int(np.round(Age_range/9))

intervals = [i for i in range(min_value, max_value+inter_value, inter_value)]
labels=[]
for i in range(1, len(intervals)):
    labels.append(i)

In [ ]:
Ames['Year_Built'] = pd.cut(x = Ames['Year_Built'], bins=intervals, labels=labels, include_lowest=True)
Ames['Year_Built']=Ames['Year_Built'].astype('int64', copy=False,errors="ignore")


In [ ]:
Age_range = Ames['Year_Remod_Add'].max() - Ames['Year_Remod_Add'].min()
min_value = int(np.floor(Ames['Year_Remod_Add'].min()))
max_value = int(np.ceil(Ames['Year_Remod_Add'].max()))

In [ ]:
# let's round the bin width
inter_value = int(np.round(Age_range/5))

intervals = [i for i in range(min_value, max_value+inter_value, inter_value)]
labels=[]
for i in range(1, len(intervals)):
    labels.append(i)


In [ ]:
Ames['Year_Remod_Add'] = pd.cut(x = Ames['Year_Remod_Add'], bins=intervals, labels=labels, include_lowest=True)
Ames['Year_Remod_Add']=Ames['Year_Remod_Add'].astype('int64', copy=False,errors="ignore")


In [ ]:
# MS_Subclass
top_5 = [x for x in Ames['MS_SubClass'].value_counts().sort_values(ascending=False).head(5).index]
for label in top_5:
    Ames[label] = np.where(Ames['MS_SubClass']==label, 1, 0)

Ames.drop("MS_SubClass", axis=1, inplace=True)
nominal.remove("MS_SubClass")


In [ ]:
# MS_Zoning
top_3 = [x for x in Ames['MS_Zoning'].value_counts().sort_values(ascending=False).head(3).index]
for label in top_3:
    Ames[label] = np.where(Ames['MS_Zoning']==label, 1, 0)

Ames.drop("MS_Zoning", axis=1, inplace=True)
nominal.remove("MS_Zoning")


In [ ]:
# Lot_Config
top_3 = [x for x in Ames['Lot_Config'].value_counts().sort_values(ascending=False).head(3).index]
for label in top_3:
    Ames[label] = np.where(Ames['Lot_Config']==label, 1, 0)

Ames.drop("Lot_Config", axis=1, inplace=True)

nominal.remove("Lot_Config")

In [ ]:
# Neighborhood 
top_8 = [x for x in Ames['Neighborhood'].value_counts().sort_values(ascending=False).head(8).index]
for label in top_8:
    Ames[label] = np.where(Ames['Neighborhood']==label, 1, 0)

Ames.drop("Neighborhood", axis=1, inplace=True)
nominal.remove("Neighborhood")

In [ ]:
# Condition_1
top_3 = [x for x in Ames['Condition_1'].value_counts().sort_values(ascending=False).head(3).index]
for label in top_3:
    Ames[label] = np.where(Ames['Condition_1']==label, 1, 0)

Ames.drop("Condition_1", axis=1, inplace=True)
nominal.remove("Condition_1")


In [ ]:
# House_Style 
top_3 = [x for x in Ames['House_Style'].value_counts().sort_values(ascending=False).head(3).index]
for label in top_3:
    Ames[label] = np.where(Ames['House_Style']==label, 1, 0)

Ames.drop("House_Style", axis=1, inplace=True)
nominal.remove("House_Style")

In [ ]:
# Roof_Style
top_2 = [x for x in Ames['Roof_Style'].value_counts().sort_values(ascending=False).head(2).index]
for label in top_2:
    Ames[label] = np.where(Ames['Roof_Style']==label, 1, 0)

Ames.drop("Roof_Style", axis=1, inplace=True)
nominal.remove("Roof_Style")

In [ ]:
# Exterior_1st 
top_5 = [x for x in Ames['Exterior_1st'].value_counts().sort_values(ascending=False).head(5).index]
for label in top_5:
    Ames[label] = np.where(Ames['Exterior_1st']==label, 1, 0)

Ames.drop("Exterior_1st", axis=1, inplace=True)
nominal.remove("Exterior_1st")

In [ ]:
# Exterior_2nd 
top_5 = [x for x in Ames['Exterior_2nd'].value_counts().sort_values(ascending=False).head(5).index]
for label in top_5:
    Ames[label] = np.where(Ames['Exterior_2nd']==label, 1, 0)

Ames.drop("Exterior_2nd", axis=1, inplace=True)
nominal.remove("Exterior_2nd")

In [ ]:
# Mas_Vnr_Type
top_3 = [x for x in Ames['Mas_Vnr_Type'].value_counts().sort_values(ascending=False).head(3).index]
for label in top_3:
    Ames[label] = np.where(Ames['Mas_Vnr_Type']==label, 1, 0)

Ames.drop("Mas_Vnr_Type", axis=1, inplace=True)
nominal.remove("Mas_Vnr_Type")

In [ ]:
# Foundation
top_3 = [x for x in Ames['Foundation'].value_counts().sort_values(ascending=False).head(3).index]
for label in top_3:
    Ames[label] = np.where(Ames['Foundation']==label, 1, 0)

Ames.drop("Foundation", axis=1, inplace=True)
nominal.remove("Foundation")

In [ ]:
# Heating
top_2 = [x for x in Ames['Heating'].value_counts().sort_values(ascending=False).head(2).index]
for label in top_2:
    Ames[label] = np.where(Ames['Heating']==label, 1, 0)

Ames.drop("Heating", axis=1, inplace=True)
nominal.remove("Heating")

In [ ]:
# Garage_Type
top_5 = [x for x in Ames['Garage_Type'].value_counts().sort_values(ascending=False).head(5).index]
for label in top_5:
    Ames[label] = np.where(Ames['Garage_Type']==label, 1, 0)

Ames.drop("Garage_Type", axis=1, inplace=True)
nominal.remove("Garage_Type")

In [ ]:
# Misc_Feature
top_2 = [x for x in Ames['Misc_Feature'].value_counts().sort_values(ascending=False).head(2).index]
for label in top_2:
    Ames[label] = np.where(Ames['Misc_Feature']==label, 1, 0)

Ames.drop("Misc_Feature", axis=1, inplace=True)
nominal.remove("Misc_Feature")

In [ ]:
# Sale_Type
top_3 = [x for x in Ames['Sale_Type'].value_counts().sort_values(ascending=False).head(3).index]
for label in top_3:
    Ames[label] = np.where(Ames['Sale_Type']==label, 1, 0)

Ames.drop("Sale_Type", axis=1, inplace=True)
nominal.remove("Sale_Type")

In [ ]:
# Sale_Condition
top_3 = [x for x in Ames['Sale_Condition'].value_counts().sort_values(ascending=False).head(3).index]
for label in top_3:
    Ames[label] = np.where(Ames['Sale_Condition']==label, 1, 0)

Ames.drop("Sale_Condition", axis=1, inplace=True)
nominal.remove("Sale_Condition")

Ames_preprocessed = pd.get_dummies(Ames)

In [ ]:
def clip_outliers(variables):
    for var in variables:
        IQR = Ames[[var]].quantile(0.75) - Ames[[var]].quantile(0.25)
        Lower_fence = float(Ames[[var]].quantile(0.25) - (IQR * 3))
        Upper_fence = float(Ames[[var]].quantile(0.75) + (IQR * 3))
        Ames[var].clip(Lower_fence, Upper_fence, inplace=True)
    return Ames

Ames=clip_outliers(continous+discrete)

In [ ]:
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

corr_features = correlation(Ames.drop(["Sale_Price"], axis=1), 0.7)
Ames.drop(labels=corr_features, axis=1, inplace=True)

In [ ]:
constant_features = [
    feat for feat in Ames.columns if Ames[feat].std() <= 0.1
]
Ames.drop(labels=constant_features, axis=1, inplace=True)
Ames=Ames.dropna()

scaler = RobustScaler().fit(Ames.drop(["Sale_Price"], axis=1))

In [ ]:
# Convert Sale_Price to log scale
Ames['Log_Sale_Price'] = np.log1p(Ames['Sale_Price'])

X = Ames.drop(['Sale_Price', 'Log_Sale_Price'], axis=1)
y = Ames['Log_Sale_Price']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=453111) 

In [ ]:
X_train.columns
# print(X_train['Alley'].unique())
# print(X_train['Street'].unique())


Index(['PID', 'Lot_Frontage', 'Lot_Area', 'Lot_Shape', 'Land_Contour',
       'Land_Slope', 'Overall_Qual', 'Overall_Cond', 'Year_Built',
       'Year_Remod_Add', 'Mas_Vnr_Area', 'Exter_Qual', 'Exter_Cond',
       'Bsmt_Qual', 'Bsmt_Cond', 'Bsmt_Exposure', 'BsmtFin_Type_1',
       'BsmtFin_Type_2', 'Bsmt_Unf_SF', 'Heating_QC', 'First_Flr_SF',
       'Second_Flr_SF', 'Bsmt_Full_Bath', 'Full_Bath', 'Bedroom_AbvGr',
       'Kitchen_Qual', 'Functional', 'Fireplaces', 'Garage_Finish',
       'Garage_Cars', 'Wood_Deck_SF', 'Open_Porch_SF', 'Fence', 'Misc_Val',
       'Mo_Sold', 'Year_Sold', 'Split_or_Multilevel', 'Split_Foyer', 'Inside',
       'Corner', 'CulDSac', 'Northwest_Ames', 'Gilbert', 'Sawyer_West',
       'Edwards', 'College_Creek', 'Crawford', 'Mitchell', 'Norm', 'PosN',
       'Gable', 'HdBoard', 'Wd Sdng', 'MetalSd', 'Plywood', 'BrkFace',
       'BrkCmn', 'WD '],
      dtype='object')

In [ ]:
# y_train.head()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, AdaBoostRegressor
import joblib
import os

# Load your data here
# X, y = ...

# Define models
models = {
    "Linear Regression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "Bagging": BaggingRegressor(),
    "AdaBoost": AdaBoostRegressor()
}

# Setup cross-validation
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

# Directory to save models
model_dir = "../AmesDashboard/models"
os.makedirs(model_dir, exist_ok=True)

# DataFrame to store RMSE
rmse_results = []

for name, model in models.items():
    train_rmse_list = []
    test_rmse_list = []
    
    for train_ix, test_ix in cv.split(X):
        # Split data
        X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
        y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]

        # Fit model
        model.fit(X_train, y_train)

        # Evaluate model
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
        train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
        train_rmse_list.append(train_rmse)
        test_rmse_list.append(test_rmse)

    # Average RMSE over all folds
    avg_train_rmse = np.mean(train_rmse_list)
    avg_test_rmse = np.mean(test_rmse_list)

    # Save model
    joblib_file = os.path.join(model_dir, f"{name.replace(' ', '_')}.joblib")
    joblib.dump(model, joblib_file)
    
    # Append RMSE results
    rmse_results.append({"Model": name, "Train RMSE": avg_train_rmse, "Test RMSE": avg_test_rmse})

# Convert to DataFrame
rmse_results_df = pd.DataFrame(rmse_results)

# Save RMSE results to JSON
rmse_file = os.path.join(model_dir, "rmse_results.json")
rmse_results_df.to_json(rmse_file, orient='records')

print("Models and RMSE results saved.")


Models and RMSE results saved.


In [ ]:
import pandas as pd
import plotly.express as px

# Load RMSE results from the JSON file
json_file = "../AmesDashboard/models/rmse_results.json"  # Adjust the path if needed
rmse_results_df = pd.read_json(json_file)

# Reshape the DataFrame for easier plotting with Plotly
rmse_melted = rmse_results_df.melt(id_vars=['Model'], 
                                   value_vars=['Train RMSE', 'Test RMSE'], 
                                   var_name='RMSE Type', value_name='RMSE')

# Plot using Plotly
fig = px.line(rmse_melted, x='Model', y='RMSE', color='RMSE Type',
              markers=True, title='RMSE Results for Different Models')

fig.update_layout(xaxis_title='Model',
                  yaxis_title='RMSE',
                  legend_title='RMSE Type',
                  legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

fig.show()
